In [7]:
import json
import urllib2
import urllib
import xmltodict
import isbntools
import os
import string
from ebooklib import epub 
from  comFunc import *
from multiprocessing import Pool
import time
import re
import functools

In [8]:
key=''.join(open("key").read().split('\n'))

In [9]:
d="/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction"
files=! find "$d" -type f -iname "*.epub"
len(files)

4584

In [11]:
#using isbntools
def getIMeta(query):
    
    print query
    isbn = app.isbn_from_words(query)    
    
        
    if isbn != None:
        meta=isbntools.app.meta(isbn)
        author=str(meta['Authors'][0])
        title=str(meta['Title'])
        fileName = author+" - "+title
        return [fileName,meta]
    else:
        return "Query Failed"

def getFileIMeta(f):
    fp,fn=os.path.split(f)
    #print(f,fp[fp.rindex('/')+1:])
    #qr=''.join(m for m in fp[fp.rindex('/')+1:] if m in string.ascii_letters+" ")
    #qr=' '.join(q for q in qr.split() if len(q)>3)
    #tm=qr.split()
    #qrr=' '.join(tm[:2]+tm[-2:])
    tmp= fn[:-5].split(" - ")
    qr=' '.join(tmp[:-1])+" by "+tmp[-1]
    print qr
    return [fn[:-5],getMeta(qr)]

In [12]:
getFileMeta(files[-1])

NameError: name 'getFileMeta' is not defined

In [ ]:
titles={}#load_obj('data/titlesIm')#{}
keys=[]#load_obj('data/titleskeysIm')#[]
t0 = time.time()
p = Pool(10)
books=p.map(
    functools.partial(getFileMeta,key=key,path="/home/some/covers/"), 
            list(filter(lambda x: x[:-5] not in keys, files[0:20]))
)
#p.join()
p.terminate()
p.close()


for book in books:
    try:
        if book[0] not in keys:
            titles[book[0]]=[book[1],book[2]]
    except:
        print(type(book))

save_obj(titles, 'data/tagsIm')
keys=keys+list(titles.keys())
save_obj(keys,'data/titleskeysIm')
t1=time.time()
total = t1-t0
print('time: ',total)

In [ ]:

#urlstring="https://www.goodreads.com/book/isbn/"+isbn+"?key="+key
urlstring="https://www.goodreads.com/search/index.xml?key="+key+"&q="+query.replace(" ","+")
print urlstring
url=urllib2.urlopen(urlstring)
url

In [29]:
data=url.read()
data

'<?xml version="1.0" encoding="UTF-8"?>\n<GoodreadsResponse>\n  <Request>\n    <authentication>true</authentication>\n      <key><![CDATA[nXDOLuXC1fR2t3N43Gxng]]></key>\n    <method><![CDATA[search_index]]></method>\n  </Request>\n  <search>\n  <query><![CDATA[The Sicilian Bought Bride by Carol Marinelli]]></query>\n    <results-start>1</results-start>\n    <results-end>2</results-end>\n    <total-results>2</total-results>\n    <source>Goodreads</source>\n    <query-time-seconds>0.15</query-time-seconds>\n    <results>\n        <work>\n  <id type="integer">7459141</id>\n  <books_count type="integer">17</books_count>\n  <ratings_count type="integer">68</ratings_count>\n  <text_reviews_count type="integer">3</text_reviews_count>\n  <original_publication_year type="integer">2004</original_publication_year>\n  <original_publication_month type="integer" nil="true"/>\n  <original_publication_day type="integer" nil="true"/>\n  <average_rating>3.34</average_rating>\n  <best_book type="Book">\n

In [30]:

data1=xmltodict.parse(data)


In [64]:
#meta=data1[u'GoodreadsResponse'][u'book']
meta=data1[u'GoodreadsResponse'][u'search'][u'results'][u'work'][0][u'best_book']
meta[u'author'][u'name']

u'Carol Marinelli'

In [67]:
img=str(meta[u'image_url'])
title=str(meta[u'title'])
author=str(meta[u'author'][u'name'])

In [68]:
urllib.urlretrieve(img,author+" - "+title+".jpg")

("Carol Marinelli - The Sicilian's Bought Bride.jpg",
 <httplib.HTTPMessage instance at 0x7f4b6d2b02d8>)

In [85]:
#query contains book name and author
# key is the goodread API key
#path is location where the cover image shall be stored
def getMeta(query,key,path):
    urlstring="https://www.goodreads.com/search/index.xml?key="+key+"&q="+query.replace(" ","+")
    print urlstring
    try:
        url=urllib2.urlopen(urlstring)
        data=xmltodict.parse(url.read())
        if len(data)>0:
            meta=data[u'GoodreadsResponse'][u'search'][u'results'][u'work'][0][u'best_book']
            dictt={}
            dictt['title']=str(meta[u'title'])
            dictt['author']=str(meta[u'author'][u'name'])
            fileName=dictt['author']+" - "+dictt['title']
            dictt['image']=path+dictt['author']+" - "+dictt['title']+".jpg"
            img=str(meta[u'image_url'])
            urllib.urlretrieve(img,dictt['image'])
            return [fileName,dictt]
        else:
            return query+" Not found"
    except:
        return query+" Not found"


In [86]:
query=""
getMeta(query,key,'/home/some/')

https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=


' Not found'

In [126]:
ls =! find /home/some/Downloads/ -type f -iname "*.html"
len(ls)

6

In [127]:
string.ascii_letters+" "

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '

In [87]:
def getFileMeta(f,key,path):

    fp,fn=os.path.split(f)
    print(f,fp[fp.rindex('/')+1:])
    qr=''.join(m for m in fp[fp.rindex('/')+1:] if m in string.ascii_letters+" ")
    qr=' '.join(q for q in qr.split() if len(q)>3)
    tm=qr.split()
    qrr=' '.join(tm[:2]+tm[-2:])
    print qrr
    return [fn[:-5],getMeta(qr,key,path)]

In [88]:
f=files[0]
getFileMeta(f,key,"/home/some/")

('/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction/Jim Butcher/Skin Game_ A Novel of the Dresden F (3482)/Skin Game_ A Novel of the Dresd - Jim Butcher.epub', 'Skin Game_ A Novel of the Dresden F (3482)')
Skin Game Novel Dresden
https://www.goodreads.com/search/index.xml?key=nXDOLuXC1fR2t3N43Gxng&q=Skin+Game+Novel+Dresden


['Skin Game_ A Novel of the Dresd - Jim Butcher',
 ['Jim Butcher - Skin Game (The Dresden Files, #15)',
  {'author': 'Jim Butcher',
   'image': '/home/some/Jim Butcher - Skin Game (The Dresden Files, #15).jpg',
   'title': 'Skin Game (The Dresden Files, #15)'}]]

In [ ]:
https://images-na.ssl-images-amazon.com/images/I/41PRV6JH3WL.jpg